# Single-cell RNA-seqs analysis using Python 

## 1. Raw Data Processing
The commands in this section are supposed to be exectued in a command-line shell terminal.  Lines that begin with a hash/pound sign are comment lines, and not meant to be run.  

Needs conda env `af`. 

### 1.1 Prepare the environment  
The environment should already have everything we need, but below is how you set up the tools.  You should have conda installed (via anaconda, miniconda, or mamba).

```
# For a LINUX machine
conda create -n af -y -c bioconda simpleaf
conda activate af
```

```
# For Apple silicon-based machine
CONDA_SUBDIR=osx-64 conda create -n af -y -c bioconda simpleaf   # create a new environment
conda activate af
conda env config vars set CONDA_SUBDIR=osx-64  # subsequent commands use intel packages

# To make your changes take effect please reactivate your environment
conda deactivate
conda activate af
```

*Dev note: Make this work, or remove `pyroe`*   
```
# It seems that pyroe is no longer a part of simpleaf, and should be installed separately.
# While inside the activated env, install pyroe via conda.
conda install -c bioconda pyroe
# Not yet sure if this will work becuase env solving seems to take a while and just doesn't complete
```

### 1.2 Get the data  
*Dev note: Test as is, but replace with data used in the Galaxy training here:*  
https://training.galaxyproject.org/training-material/topics/single-cell/tutorials/scrna-case_alevin/tutorial.html   
The subset is originally from [E-MTAB-6945](https://www.ebi.ac.uk/biostudies/arrayexpress/studies/E-MTAB-6945)

```
# Create a working dir and go to the working directory
## The && operator helps execute two commands using a single line of code.
mkdir ex_xmpl_run && cd ex_xmpl_run

# Fetch the example dataset
## Set destination directory of fastq files as `toy_read_fastq`.
fastq_dir="toy_read_fastq"
mkdir $fastq_dir
## Set destination directory of reference files as `toy_mouse_ref`.
ref_dir="toy_mouse_ref"
mkdir $ref_dir

wget -q -P $fastq_dir https://zenodo.org/record/4574153/files/SLX-7632.TAAGGCGA.N701.s_1.r_1.fq-400k.fastq
wget -q -P $fastq_dir https://zenodo.org/record/4574153/files/SLX-7632.TAAGGCGA.N701.s_1.r_2.fq-400k.fastq

# Fetch CB permit list
## No permit list for dropseq scRNA-seq data (we use a cutoff in alevin)
```

### 1.3 Generate a single-cell matrix with `simpleaf`
*Dev note: Test as is, recommend the Galaxy training here for a more detailed hands-on:*  
https://training.galaxyproject.org/training-material/topics/single-cell/tutorials/scrna-case_alevin/tutorial.html

```
# simpleaf needs the environment variable ALEVIN_FRY_HOME to store configuration and data.
# For example, the paths to the underlying programs it uses and the CB permit list
mkdir alevin_fry_home & export ALEVIN_FRY_HOME='alevin_fry_home'

# the simpleaf set-paths command finds the path to the required tools and write a configuration JSON file in the ALEVIN_FRY_HOME folder.
simpleaf set-paths

# simpleaf index
# Usage: simpleaf index -o out_dir [-f genome_fasta -g gene_annotation_GTF|--refseq transcriptome_fasta] -r read_length -t number_of_threads
## The -r read_lengh is the number of sequencing cycles performed by the sequencer to generate biological reads (100 bp for this dataset).
## Publicly available datasets usually have the read length in the description. Sometimes they are called the number of cycles.
simpleaf index \
-o simpleaf_index \
-f toy_mouse_ref/mus_musculus_genome.fa \
-g toy_mouse_ref/mus_musculus_genes.gtf \
-r 100 \
-t 8
```

Above `simpleaf index` command should run for a few seconds.  
Standard output is something like below.  

```
(af)[fg_atlas@hl-codon-22-01 ex_xmpl_run]$ simpleaf index -o simpleaf_index -f toy_mouse_ref/mus_musculus_genome.fa -g toy_mouse_ref/mus_musculus_genes.gtf -r 100 -t 8
2023-08-24T15:41:52.482355Z  INFO simpleaf::simpleaf_commands::indexing: preparing to make reference with roers
2023-08-24T15:41:59.172897Z  INFO grangers::reader::gtf: Finished parsing the input file. Found 5 comments and 889667 records.
2023-08-24T15:42:00.587437Z  INFO roers: Built the Grangers object for 889667 records
2023-08-24T15:42:01.382828Z  WARN roers: Found missing gene_id and/or gene_name; Imputing. If both missing, will impute using transcript_id; Otherwise, will impute using the existing one.
2023-08-24T15:42:01.409041Z  INFO roers: Proceed 408411 exon records from 69229 transcripts
2023-08-24T15:42:21.479623Z  INFO roers: Processing 339182 intronic records
2023-08-24T15:42:43.410348Z  INFO roers: Done!
2023-08-24T15:42:43.778421Z  INFO simpleaf::simpleaf_commands::indexing: salmon index cmd : /hps/nobackup/irene/ma/isl/envs_test/af/bin/salmon index -k 31 -i simpleaf_index/index -t simpleaf_index/ref/roers_ref.fa --threads 8
2023-08-24T15:48:28.997354Z  INFO simpleaf::utils::prog_utils: command returned successfully (exit status: 0)
```

```
# Collecting sequencing read files (if there is more than one pair)
# simpleaf quant
## Usage: simpleaf quant -c chemistry -t threads -1 reads1 -2 reads2 -i index -u [unspliced permit list] -r resolution -m t2g_3col -o output_dir
simpleaf quant \
-c "1{b[12]u[8]x:}2{r:}" \
-t 8 \
-1 $fastq_dir/SLX-7632.TAAGGCGA.N701.s_1.r_1.fq-400k.fastq \
-2 $fastq_dir/SLX-7632.TAAGGCGA.N701.s_1.r_2.fq-400k.fastq \
-i simpleaf_index/index \
-f 100000 \
-d fw \
-r cr-like \
-m simpleaf_index/index/t2g_3col.tsv \
-o simpleaf_quant
```

Above `simpleaf quant` command should take a few seconds to run.  
Standard output is something like below.

```
(af)[fg_atlas@hl-codon-121-03 ex_xmpl_run]$ simpleaf quant -c "1{b[12]u[8]x:}2{r:}" -t 8 -1 $fastq_dir/SLX-7632.TAAGGCGA.N701.s_1.r_1.fq-400k.fastq -2 $fastq_dir/SLX-7632.TAAGGCGA.N701.s_1.r_2.fq-400k.fastq -i simpleaf_index/index -f 100000 -d fw -r cr-like -m simpleaf_index/index/t2g_3col.tsv -o simpleaf_quant
2023-08-25T14:12:03.800114Z  INFO simpleaf::simpleaf_commands::quant: salmon alevin cmd : /hps/nobackup/irene/ma/isl/envs_test/af/bin/salmon alevin --index simpleaf_index/index -l A -1 toy_read_fastq/SLX-7632.TAAGGCGA.N701.s_1.r_1.fq-400k.fastq -2 toy_read_fastq/SLX-7632.TAAGGCGA.N701.s_1.r_2.fq-400k.fastq --read-geometry 2[1-end] --bc-geometry 1[1-12] --umi-geometry 1[13-20] --threads 8 -o simpleaf_quant/af_map --sketch
2023-08-25T14:12:10.295563Z  INFO simpleaf::utils::prog_utils: command returned successfully (exit status: 0)
2023-08-25T14:12:10.295601Z  INFO simpleaf::simpleaf_commands::quant: alevin-fry generate-permit-list cmd : /hps/nobackup/irene/ma/isl/envs_test/af/bin/alevin-fry generate-permit-list -i simpleaf_quant/af_map -d fw --force-cells 100000 -o simpleaf_quant/af_quant
2023-08-25T14:12:11.282405Z  INFO simpleaf::utils::prog_utils: command returned successfully (exit status: 0)
2023-08-25T14:12:11.282435Z  INFO simpleaf::simpleaf_commands::quant: alevin-fry collate cmd : /hps/nobackup/irene/ma/isl/envs_test/af/bin/alevin-fry collate -i simpleaf_quant/af_quant -r simpleaf_quant/af_map -t 8
2023-08-25T14:12:14.922569Z  INFO simpleaf::utils::prog_utils: command returned successfully (exit status: 0)
2023-08-25T14:12:14.922596Z  INFO simpleaf::simpleaf_commands::quant: cmd : "/hps/nobackup/irene/ma/isl/envs_test/af/bin/alevin-fry" "quant" "-i" "simpleaf_quant/af_quant" "-o" "simpleaf_quant/af_quant" "-t" "8" "-m" "simpleaf_index/index/t2g_3col.tsv" "-r" "cr-like"
2023-08-25T14:12:17.751579Z  INFO simpleaf::utils::prog_utils: command returned successfully (exit status: 0)
```

View the resulting files. Commands are preceded by `$` (the dollar sign should not be copied), and results are shown right below the commands.

```
# Each line in `quants_mat.mtx` represents
# a non-zero entry in the format row column entry
# values you see may be different
$ tail -3 simpleaf_quant/af_quant/alevin/quants_mat.mtx
61328 18849 1
61329 9312 1
61330 10839 1

# Each line in `quants_mat_cols.txt` is a splice status
# of a gene in the format (gene name)-(splice status)
# values you see may be different
$ tail -3 simpleaf_quant/af_quant/alevin/quants_mat_cols.txt
ENSMUSG00002075482-A
ENSMUSG00000117941-A
ENSMUSG00000117659-A

# Each line in `quants_mat_rows.txt` is a corrected
# (and, potentially, filtered) cell barcode
# values you see may be different
$ tail -3 simpleaf_quant/af_quant/alevin/quants_mat_rows.txt
AGGAACACATAG
CCTAAGACGGTT
CGCCTTAGGGCC
```

*Dev note: Check if this is necessary. If not and pyroe cannot be installed, scrap below*

In [ ]:
# import pyroe

# quant_dir = 'simpleaf_quant/af_quant'
# adata_sa = pyroe.load_fry(quant_dir)

In [ ]:
# import pyroe

# quant_dir = 'simpleaf_quant/af_quant'
# adata_usa = pyroe.load_fry(quant_dir, output_format={'X' : ['U','S','A']})

### 1.4 Generate a single-cell matrix with the `alevin-fry` pipeline

#### 1.4.1 Using the SC book method

Build the reference index

```
# make splici reference
## Usage: pyroe make-splici genome_file gtf_file read_length out_dir
## The read_length is the number of sequencing cycles performed by the sequencer. Ask your technician if you are not sure about it.
## Publicly available datasets usually have the read length in the description.
pyroe make-splici \
$ref_dir/mus_musculus_genome.fa \
$ref_dir/mus_musculus_genes.gtf \
98 \
splici_rl98_ref

# Index the reference
## Usage: salmon index -t extend_txome.fa -i idx_out_dir -p num_threads
## The $() expression runs the command inside and puts the output in place.
## Please ensure that only one file ends with ".fa" in the `splici_ref` folder.
salmon index \
-t $(ls splici_rl98_ref/*\.fa) \
-i salmon_index \
-p 8
```

Above commands should take seconds each to run.  
To execute commands, had to switch between different conda envs because `pyroe` doesn't get installed with `simpleaf` via conda.

Mapping and quantification

```
# Mapping
## Usage: salmon alevin -i index_dir -l library_type -1 reads1_files -2 reads2_files -p num_threads -o output_dir
salmon alevin \
-i salmon_index \
-l ISR \
-1 $fastq_dir/SLX-7632.TAAGGCGA.N701.s_1.r_1.fq-400k.fastq \
-2 $fastq_dir/SLX-7632.TAAGGCGA.N701.s_1.r_2.fq-400k.fastq \
-p 8 \
-o salmon_alevin \
--dropseq \
--sketch
```

```
# Cell barcode correction
## Usage: alevin-fry generate-permit-list -u CB_permit_list -d expected_orientation -o gpl_out_dir
## Here, the reads that map to the reverse complement strand of transcripts are filtered out by specifying `-d fw`.
alevin-fry generate-permit-list \
-d fw \
-f 100000 \
-i salmon_alevin \
-o alevin_fry_gpl
```

Output of generating permit list

```
2023-08-27 16:48:27 INFO paired : false, ref_count : 98,778, num_chunks : 45                                                          
2023-08-27 16:48:27 INFO read 2 file-level tags
2023-08-27 16:48:27 INFO read 2 read-level tags
2023-08-27 16:48:27 INFO read 1 alignemnt-level tags
2023-08-27 16:48:27 INFO File-level tag values FileTags { bclen: 12, umilen: 8 }                                                      
2023-08-27 16:48:27 INFO observed 202,085 reads in 45 chunks --- max ambiguity read occurs in 2,266 refs                              
2023-08-27 16:48:28 INFO total number of distinct corrected barcodes : 61,328
```

```
# Filter mapping information
## Usage: alevin-fry collate -i gpl_out_dir -r alevin_map_dir -t num_threads
alevin-fry collate \
-i alevin_fry_gpl \
-r salmon_alevin \
-t 8
```

Output of collate:  

```
2023-08-27 18:12:40 INFO filter_type = Filtered
2023-08-27 18:12:40 INFO collated rad file will not be compressed
2023-08-27 18:12:40 INFO paired : false, ref_count : 98,778, num_chunks : 45, expected_ori : Forward
2023-08-27 18:12:40 INFO read 2 file-level tags
2023-08-27 18:12:40 INFO read 2 read-level tags
2023-08-27 18:12:40 INFO read 1 alignemnt-level tags
2023-08-27 18:12:40 INFO File-level tag values FileTags { bclen: 12, umilen: 8 }
2023-08-27 18:12:44 INFO deserialized correction map of length : 4,361,264
2023-08-27 18:12:44 INFO Generated 1 temporary buckets.
  [00:00:00] [╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢]      45/45      partitioned records into temporary files.
  [00:00:00] [╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢]       1/1       gathered all temp files.
2023-08-27 18:12:44 INFO writing num output chunks (61,328) to header
2023-08-27 18:12:44 INFO expected number of output chunks 61,328
2023-08-27 18:12:44 INFO finished collating input rad file "salmon_alevin/map.rad".
```

```
# UMI resolution + quantification
## Usage: alevin-fry quant -r resolution -m txp_to_gene_mapping -i gpl_out_dir -o quant_out_dir -t num_threads
## The file ends with `3col.tsv` in the splici_ref folder will be passed to the -m argument.
## Please ensure that there is only one such file in the `splici_ref` folder.
alevin-fry quant -r cr-like-em \
-m $(ls splici_rl98_ref/*3col.tsv) \
-i alevin_fry_gpl \
-o alevin_fry_quant \
-t 8
```

Output of above command:  

```
2023-08-27 18:16:01 INFO quantifying from uncompressed, collated RAD file File { fd: 4, path: "/homes/irisyu/training/scrnaseq_python_2023/ex_xmpl_run/alevin_fry_gpl/map.collated.rad", read: true, write: false }
2023-08-27 18:16:01 INFO paired : false, ref_count : 98,778, num_chunks : 61,328
2023-08-27 18:16:01 INFO tg-map contained 25,810 genes mapping to 98,778 transcripts.
2023-08-27 18:16:01 INFO read 2 file-level tags
2023-08-27 18:16:01 INFO read 2 read-level tags
2023-08-27 18:16:01 INFO read 1 alignemnt-level tags
2023-08-27 18:16:01 INFO File-level tag values FileTags { bclen: 12, umilen: 8 }
  [00:00:02] [╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢╢]   61328/61328   finished quantifying 61,328 cells.
2023-08-27 18:16:04 INFO processed 180,774 total read records
```

#### 1.4.2 Using the Atlas pipeline  
Try and replicate the Atlas pipeline here. Below are the references:  
[Tutorial](https://training.galaxyproject.org/training-material/topics/single-cell/tutorials/scrna-case_alevin/tutorial.html)  
[Galaxy workflow](https://humancellatlas.usegalaxy.eu/published/workflow?id=bd7bd54eaf932585)

#### Convert from mtx to 10x  
Below is a script that also uses pyroe, with USA mode  (use conda env `pyroe` or `parse_alevin_fry` in codon to run).
In codon, the script is copied to `/homes/irisyu/training/scrnaseq_python_2023/bin` (add to PATH).

export PATH=/homes/irisyu/training/scrnaseq_python_2023/bin:$PATH

```
(pyroe) % alevinFryMtxTo10x.py alevin_fry_quant alevin_fry_parsed single_cell 
USA mode: True
Using pre-defined output format: scrna
Will populate output field X with sum of counts frorm ['S', 'A'].
```

#### Remove empty drops  

Below, use conda env with `bioconductor-dropletutils` and `dropletutils-scripts`.  In codon, the scripts `/nfs/production/irene/ma/fg_atlas_sc/nextflow_scxa_test/envs/dropletutils-a2f6101409ff7f11d8d66c59f1f32ead/bin/dropletutils-*.R` are copied to `/homes/irisyu/training/scrnaseq_python_2023/bin` (add to PATH).  The scripts should also be in the dropletutils-scripts env bin.  

```
(dropletutils)[fg_atlas@hl-codon-46-03 ex_xmpl_run]$ dropletutils-read-10x-counts.R -s alevin_fry_parsed -c TRUE -o alevin_fry_parsed/matrix.rds
..
# Object summary
class: SingleCellExperiment
dim: 25810 61328
metadata(0):
assays(1): counts
rownames(25810): ENSMUSG00000026315 ENSMUSG00000063558 ...
  ENSMUSG00002075994 ENSMUSG00002075332
rowData names(2): ID Symbol
colnames(61328): AGATGTGCGGTC CTAAACTACTCG ... ATGACATTCAGG
  CACCGGGGGAGG
colData names(2): Sample Barcode
reducedDimNames(0):
spikeNames(0):

# Metadata sample
DataFrame with 6 rows and 2 columns
                        Sample      Barcode
                   <character>  <character>
AGATGTGCGGTC alevin_fry_parsed AGATGTGCGGTC
CTAAACTACTCG alevin_fry_parsed CTAAACTACTCG
GTGTGACGTGTG alevin_fry_parsed GTGTGACGTGTG
AGTGGCTTACGA alevin_fry_parsed AGTGGCTTACGA
GCCGATTTTTAC alevin_fry_parsed GCCGATTTTTAC
CGGAACCCCACT alevin_fry_parsed CGGAACCCCACT


(dropletutils)[fg_atlas@hl-codon-101-03 af_xmpl_run]$ mkdir empty_drops
(dropletutils)[fg_atlas@hl-codon-46-03 ex2_xmpl_run]$ dropletutils-empty-drops.R -i alevin_fry_parsed/matrix.rds --lower 5 --niters 100
0 --filter-empty TRUE --filter-fdr 0.01 -o empty_drops/nonempty.rds -t empty_drops/nonempty.txt                                       
...
At an FDR of 0.01, estimate that 23 barcodes have cells.
Will filter to 23 barcodes.

Parameter values:
             value
lower            5
niters        1000
test_ambient FALSE
filter_empty  TRUE
filter_fdr    0.01
```

#### Convert from rds to h5ad using sceasy

Below, use conda env with `r-sceasy` (e.g., `sc_py_training`).  

```
(sc_py_training)[fg_atlas@hl-codon-46-03 ex2_xmpl_run]$ Rscript -e 'library(sceasy)' \
> -e 'sce <- readRDS("empty_drops/nonempty.rds")' \
> -e 'sceasy::convertFormat(sce, outFile="empty_drops/nonempty.h5ad", from="sce", to="anndata", main_layer="counts")' \
> -e 'print(sce)'
During startup - Warning message:
Setting LC_CTYPE failed, using "C"
Loading required package: reticulate
Loading required namespace: SingleCellExperiment
/hps/nobackup/irene/ma/isl/envs_test/sc_py_training/lib/R/library/reticulate/python/rpytools/loader.py:117: FutureWarning: pandas.core$
index is deprecated and will be removed in a future version. The public classes are available in the top-level namespace.
  return _find_and_load(name, import_)
/hps/software/users/ma/service/isl/conda/config/envs/r-anndata/lib/python3.11/site-packages/anndata/core/anndata.py:1328: FutureWarnin$
: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  and not is_categorical(df[key])
/hps/software/users/ma/service/isl/conda/config/envs/r-anndata/lib/python3.11/site-packages/anndata/core/anndata.py:1328: FutureWarnin$
: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  and not is_categorical(df[key])
/hps/software/users/ma/service/isl/conda/config/envs/r-anndata/lib/python3.11/site-packages/anndata/core/anndata.py:101: FutureWarning$
 is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  if is_string_dtype(df[k]) and not is_categorical(df[k]):
/hps/software/users/ma/service/isl/conda/config/envs/r-anndata/lib/python3.11/site-packages/anndata/core/anndata.py:108: FutureWarning$
 is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  elif is_categorical(df[k]):
AnnData object with n_obs <U+00D7> n_vars = 23 <U+00D7> 25810
    obs: 'Barcode', 'emptyTotal', 'emptyLogProb', 'emptyPValue', 'emptyLimited'
    var: 'ID', 'Symbol'
Warning message:
In .regularise_df(as.data.frame(SummarizedExperiment::colData(obj)),  :
  Dropping single category variables:Sample, emptyFDR
class: SingleCellExperiment
dim: 25810 23
metadata(0):
assays(1): counts
rownames(25810): ENSMUSG00000026315 ENSMUSG00000063558 ...
  ENSMUSG00002075994 ENSMUSG00002075332
rowData names(2): ID Symbol
colnames(23): GATGCATTCCTG CCTTCCTCCTTC ... GACGAGTGTCCC CCATTCCTGTCT
colData names(7): Sample Barcode ... emptyLimited emptyFDR
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):
```

#### Questions  
1. How many cell barcodes do you start with?  
2. How many cell barcodes are left after the emptyDrops treatment?  

### Exercises
Answer the Questions in each section above. 

### 2.  Quality Control  

#### 2.1 Filtering low quality reads

In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
from scipy.stats import median_abs_deviation

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)

workdir = "/home/training/scrnaseq_python_2023"

In [ ]:
# For exercises, we are using the raw counts of E-MTAB-6945 (not the subset)
adata = sc.read(f"{workdir}/data/E-MTAB-6945.h5ad")
adata

In [ ]:
# No need to make var names unique, since all are unique
# adata.var_names_make_unique()
adata

The dataset has 5,218 barcodes and 35,682 transcripts.

Below, we add series of booleans to identify whicy variables (transcripts) are from mitochondrial, ribosomal, or hemoglobin genes.

In [ ]:
# Mitochondrial transcripts already annotated
adata.var.loc[adata.var.chromosome == "MT", ["gene_symbols", "chromosome", "mito"]] 

Below, we calculate qc metrics, which will evaluate the raw observations and therefore will add data to the `adata.obs` dataframe. 

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mito"], inplace=True, percent_top=[20], log1p=True
)
adata

In the third plot below, show the log-transformed versions of the `total_counts` and `n_genes_by_counts` in their respective axes.

In [ ]:
# Ditribution of barcodes (cells) in terms of the numbr of genes per barcode (cell)
p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
# Above distribution viewed as a violon plot:
# sc.pl.violin(adata, 'total_counts')

# Distribution of barcodes (cells) in terms of the %mt of counts per barcode (cell)
p2 = sc.pl.violin(adata, "pct_counts_mito")

# A scatter representing total_counts (x), n_genes_by_counts (y), and %mt of counts (color) per barcode (cell)
p3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mito")

In [ ]:
adata.obs.loc[:, ['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 
                  'log1p_total_counts', 'pct_counts_in_top_20_genes']
]

Below defines a function that checks if of an observation is an outlier based on nmads * MAD.  The output is a series of boleans.  
`nmads` or "number of MADs" refers to the distance from the median in terms of MADs, outside of which a value is considered an outlier.  
Note that MAD will always be positive.

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

Below: `|` is a bitwise or operator comparing a 3 serieses of booleans (metrics).  If an observation (barcode) is an outlier in any of the three metrics, that barcode is tagged as an outlier.

In [ ]:
adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
adata.obs.outlier.value_counts()

Below, observations (barcodes) are tagged as outliers based on metrics  for the presence of mitochondrial transcripts.

In [ ]:
adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mito", 3) | (
    adata.obs["pct_counts_mito"] > 8
)
adata.obs.mt_outlier.value_counts()

Use outlier information to filter barcodes (cells).  Below, we use `~` which is a bitwise `not`, an inverse boolean mask (`True` to `False` and vice versa).

In [ ]:
print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()
print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")

Edit the plotting command below, so that the plot shows the log-transformed versions of the `total_counts` and `n_genes_by_counts` in their respective axes. Compare with the plot above.

In [ ]:
p4 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mito")

#### 

#### 2.2 Correction of ambient RNA 

In [ ]:
import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
%%R
library(SoupX)

In [ ]:
# Normalise and log1p-transform the data
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp)
sc.pp.log1p(adata_pp)

In [ ]:
# Compute principal components
sc.pp.pca(adata_pp)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="soupx_groups")

# Preprocess variables for SoupX
soupx_groups = adata_pp.obs["soupx_groups"]

In [ ]:
del adata_pp

In [ ]:
cells = adata.obs_names
genes = adata.var_names
data = adata.X.T

In [ ]:
# Load the raw again, required by
adata_raw = sc.read(f"{workdir}/data/E-MTAB-6945.h5ad")
data_tod = adata_raw.X.T

In [ ]:
del adata_raw

In [ ]:
%%R -i data -i data_tod -i genes -i cells -i soupx_groups -o out 

# specify row and column names of data
rownames(data) = genes
colnames(data) = cells
# ensure correct sparse format for table of counts and table of droplets
data <- as(data, "sparseMatrix")
data_tod <- as(data_tod, "sparseMatrix")

# Generate SoupChannel Object for SoupX 
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

# Add extra meta data to the SoupChannel object
soupProf = data.frame(row.names = rownames(data), est = rowSums(data)/sum(data), counts = rowSums(data))
sc = setSoupProfile(sc, soupProf)
# Set cluster information in SoupChannel
sc = setClusters(sc, soupx_groups)

# Estimate contamination fraction
sc  = autoEstCont(sc, doPlot=FALSE)
# Infer corrected table of counts and rount to integer
out = adjustCounts(sc, roundToInt = TRUE)

In [ ]:
type(out)

In [ ]:
# Move counts to a layer
adata.layers["counts"] = adata.X

# Put the output of soupx into a layer
adata.layers["soupX_counts"] = out.T

# Use the soupx layer as the new main data layer, X
adata.X = adata.layers["soupX_counts"]

In [ ]:
print(f"Total number of genes: {adata.n_vars}")

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print(f"Number of genes after cell filter: {adata.n_vars}")

The QC steps above filter the data into what can be considered the real RNAs.

Note: Above count for number of genes is not the same as the book. 

In [ ]:
# Save above data, so as not to repeat all of above when kernel stops
# adata.write("E-MTAB-6945_corrected_ambient_RNA.h5ad")

#### 2.3 Doublet Detection  

In [ ]:
%%R
library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)

In [ ]:
data_mat = adata.X.T

In [ ]:
%%R -i data_mat -o doublet_score -o doublet_class

set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

In [ ]:
adata.obs["scDblFinder_score"] = doublet_score
adata.obs["scDblFinder_class"] = doublet_class
adata.obs.scDblFinder_class.value_counts()

In [ ]:
# adata.write("E-MTAB-6945_quality_control.h5ad")

### 3. Normalization

In [ ]:
import scanpy as sc
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import anndata2ri
import logging
from scipy.sparse import issparse

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    # color_map="YlGnBu",
    frameon=False,
)

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
adata = sc.read("E-MTAB-6945_quality_control.h5ad")
adata

In [ ]:
p1 = sns.histplot(adata.obs["total_counts"], bins=100, kde=False)

#### 3.1 Shifted logarithm

In [ ]:
scales_counts = sc.pp.normalize_total(adata, target_sum=None, inplace=False)
# log1p transform
adata.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
p1 = sns.histplot(adata.obs["total_counts"], bins=100, kde=False, ax=axes[0])
axes[0].set_title("Total counts")
p2 = sns.histplot(adata.layers["log1p_norm"].sum(1), bins=100, kde=False, ax=axes[1])
axes[1].set_title("Shifted logarithm")
plt.show()

In [ ]:
from scipy.sparse import csr_matrix, issparse

In [ ]:
%%R
library(scran)
library(BiocParallel)

In [ ]:
# Preliminary clustering for differentiated normalisation
adata_pp = adata.copy()
sc.pp.normalize_total(adata_pp)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="groups")

In [ ]:
data_mat = adata_pp.X.T
# convert to CSC if possible. See https://github.com/MarioniLab/scran/issues/70
if issparse(data_mat):
    if data_mat.nnz > 2**31 - 1:
        data_mat = data_mat.tocoo()
    else:
        data_mat = data_mat.tocsc()
ro.globalenv["data_mat"] = data_mat
ro.globalenv["input_groups"] = adata_pp.obs["groups"]

In [ ]:
del adata_pp

In [ ]:
%%R -o size_factors

size_factors = sizeFactors(
    computeSumFactors(
        SingleCellExperiment(
            list(counts=data_mat)), 
            clusters = input_groups,
            min.mean = 0.1,
            BPPARAM = MulticoreParam()
    )
)

In [ ]:
adata.obs["size_factors"] = size_factors
scran = adata.X / adata.obs["size_factors"].values[:, None]
adata.layers["scran_normalization"] = csr_matrix(sc.pp.log1p(scran))

In [ ]:
# adata.write("E-MTAB-6945_log1p_normalization.h5ad")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
p1 = sns.histplot(adata.obs["total_counts"], bins=100, kde=False, ax=axes[0])
axes[0].set_title("Total counts")
p2 = sns.histplot(
    adata.layers["scran_normalization"].sum(1), bins=100, kde=False, ax=axes[1]
)
axes[1].set_title("log1p with Scran estimated size factors")
plt.show()

#### 3.2 Analytic Pearson Residuals

In [ ]:
# Run without comment below if kernel stops
# adata = sc.read("E-MTAB-6945_log1p_normalization.h5ad")

In [ ]:
analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(adata, inplace=False)
adata.layers["analytic_pearson_residuals"] = csr_matrix(analytic_pearson["X"])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
p1 = sns.histplot(adata.obs["total_counts"], bins=100, kde=False, ax=axes[0])
axes[0].set_title("Total counts")
p2 = sns.histplot(
    adata.layers["analytic_pearson_residuals"].sum(1), bins=100, kde=False, ax=axes[1]
)
axes[1].set_title("Analytic Pearson residuals")
plt.show()

In [ ]:
# adata.write("E-MTAB-6945_normalization.h5ad")

### 4. Feature Selection

In [ ]:
import scanpy as sc
import anndata2ri
import logging
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
%%R
library(scry)

In [ ]:
adata = sc.read("E-MTAB-6945_normalization.h5ad")

In [ ]:
ro.globalenv["adata"] = adata

In [ ]:
%%R
sce = devianceFeatureSelection(adata, assay="X")

In [ ]:
binomial_deviance = ro.r("rowData(sce)$binomial_deviance").T

In [ ]:
idx = binomial_deviance.argsort()[-4000:]
mask = np.zeros(adata.var_names.shape, dtype=bool)
mask[idx] = True

adata.var["highly_deviant"] = mask
adata.var["binomial_deviance"] = binomial_deviance

In [ ]:
sc.pp.highly_variable_genes(adata, layer="scran_normalization")

In [ ]:
ax = sns.scatterplot(
    data=adata.var, x="means", y="dispersions", hue="highly_deviant", s=5
)
ax.set_xlim(None, 1.5)
ax.set_ylim(None, 3)
plt.show()

In [ ]:
# adata.write("E-MTAB-6945_feature_selection.h5ad")